In [ ]:
import sys
sys.path.append("/Users/jlazar/research/IceCube_Masterclass_MoonShadow/")

# Zenith and Azimuth

To talk about which direction some is coming from, we usually think about this in terms of two angles, the zenith and azimuth. If you are familiar with spherical coordinate systems then you may remember these angle, but if not no worries we'll give you a quick rundown !

You can think of looking up and and down as looking in different _zenith_ directions. If you looked straight up, you would be looking at zenith of $0^{\circ}$ and straight ahead a zenith of $90^{\circ}$. Looking left or right allows you to look at different azimuths. If we define looking straight ahead as azimuth of $0^{\circ}$, the when we looked over our left (right) shoulders, we would be looking at an azimuth of $90^{\circ}$ ($-90^{\circ}$).

You can play around with the following little diagram that shows how changing the zenith or azimuth changes the direction vector to build up some intuition for these. Can you try to get the vector to point to Cambridge ?

In [ ]:
# earth coordinates:
long = -90.0
lat = -90.0

# neutrino direction coordinates:
azi = 60.0
zen = 135.0

plot_direction_from_earth(lat=lat, long=long, azi=azi, zen=zen)

# Where is the Moon ?

The first thing that we will need to be able to calculate is the position of the Moon at given times. We have created a `Moon` object that has a `position` method. You can call this with a date given in a few different formats. First, you can give a datetime `str` in the form `"YYYY-MM-DD[-HH-MM-SS]"`. Also you can provide this as a Modified Julian Date (MJD). While this may sound scary, it is just a way of representing a date as a number, with each day being one integer increment. Today is MJD 60,069 ! Obviously, this will depend on the latitude and longitude that we are at. Let's specify Cambridge's latitude and longitude !

In [ ]:
from analysis import moon

cambridge_lat = 45.0
cambridge_long = 30.0

t = "2023-05-21-12-00-00" # Today at noon !
zen, az = moon.position(t, lat=cambridge_lat, lon=cambridge_long)
print(f"At {t} the Moon was at zenith {zen} degrees and azimuth {az} degrees")

t = 60_085.5 # Also today at noon !
zen, az = moon.position(t, lat=cambridge_lat, lon=cambridge_long)
print(f"At {t} MJD the Moon was at zenith {zen} degrees and azimuth {az} degrees")

## Now we can ask what happens as the Moon moves through the sky for many days. you can fill in your own start and end date ! See what happens when you change the range from months scale to years !

In [ ]:
import numpy as np
from datetime import datetime
from tqdm.notebook import tqdm

start_date = "2010-11-14"
# Fill out an end date !
end_date = "2023-05-04"
# end_date = 

dates = np.arange(start_date, end_date, dtype='datetime64[D]')

moon_zeniths = []
moon_azimuths = []

# Now we iterate over all the dates, and calculate the moon position for each.
# At each step, we will `append` the zenith and azimuth to our lists
for idx, date in tqdm(enumerate(dates)):
    
    # Call `moon.position` to find it's direction
    zen, az = 
    
    moon_zeniths.append(zen)
    
    # Put the azimuth in the correct array :-)

## Let's make some plots of this and see if there is anything interesting.

In [ ]:
import matplotlib.pyplot as plt

# Make a plot of the azimuthal angle !
fig, ax = plt.subplots(figsize=(10,5))
plt.plot(dates, np.degrees(moon_azimuths))
plt.xlabel("Date")
plt.ylabel(r"Azimuth angle [$^\circ$]")
plt.show()

# Your turn to make the zenith plot.
# Don't forget to label you axes !!!

## These are fine if perhaps a bit underwhelming, but we can visualize this in a more intuitive way. A _sky map_ helps us visualize all the sky at once. Let's plot try that out !

In [ ]:
from analysis import plot_skymap
plot_skymap(moon_zeniths, moon_azimuths)

## Before moving along, take a second to turn some of these knobs and see what happens ! The best way to build up intuition is fiddling around and seeing what happens :-)

# Reconstructing the event directions !

If we want to find out where a neutrino came from, we must attempt to reconstruct the direction from the light we saw in the detector. In the following, we will 

In [ ]:
from analysis import DataReader
tracks = DataReader("tracks.parquet")

In [ ]:
from analysis import reco_game
reco_game(tracks)

### Now we can do the same but for another class of events, cascades.

In [ ]:
cascades = DataReader("cascades.parquet")
reco_game(cascades)

### Which of these _morphologies_ did you find it easier to guess the direction for ? What does this tell you about what kind of events one should use for an analysis where we want to point to a specific object ?

## Let's see if we can see the Moon !

First, we will need to compute the position of the Moon when we observed each event. The tracks have a field `time` where we give the time we saw the event. Combine this with our `moon` position method to find the position of the Moon when we saw each event.

In [ ]:
# from tqdm.notebook import tqdm
moon_zeniths = []
moon_azimuths = []

for time in tqdm(tracks["time"]):
    
    zen, az = 
    # Now `append` to the proper lists
    

In [ ]:
delta_zeniths = []
for zenith, moon_zenith in zip(tracks["zenith"], moon_zeniths):
    delta_zenith = zenith - moon_zenith
    delta_zeniths.append(delta_zenith)

## Can you do the same for azimuths ?

There is a subtlety here that comes from the periodic nature of the azimuth anlge. Do you see what this is ? Can you think of a way to address this ? Maybe discuss with other students or talk to one of the instructors.

In [ ]:
# This will be blank in the final version.
delta_azimuths = []

for azimuth, moon_azimuth in zip(tracks["azimuth"], moon_azimuths):
    delta_az = 

### Let's plot a histogram of the angular distance from the center of the Moon.

We will use a histogram for this, which is one of particle physicists favorite tools for looking at data. It will tell you how many events fall within a certain range of values. Experiment with different number of bins in each direction, and try to see if you can understand the tradeoff of having either too many bins or too few. If you want to personalize your plot, you can change the colormap that is used by changing the `cmap` keyword argument. You can find a loit of allowed colormaps [here](https://matplotlib.org/stable/tutorials/colors/colormaps.html). 

Also, what is that `sine` doing there ? Bonus points if you can figure it out...

In [ ]:
from analysis import sine, make_plot
import numpy as np

minimum_angle = -5 # degrees
maximum_angle = 5 # degrees
n_bins = 

angle_bins = np.linspace(minimum_angle, maximum_angle, n_bins+1)

h, _, _ = np.histogram2d(
    sine(tracks["zenith"]) * delta_azimuths,
    delta_zeniths,
    bins=[angle_bins, angle_bins]
)

make_plot(h, angle_bins, cmap="cool_r")

## Investigation into the dependence on energy

By running the following cell, you will see the energy of each of the neutrinos that gave rise to the events in the reconstruction game. We'll then play the game again and try to see if any patterns jump out.

In [ ]:
event_list = [24, 18973, 25456, 80336, 18901, 172, 47547, 5717, 831, 696]
energies = []
# go over each event_id in the event_list
for idx, event_id in enumerate(event_list):
    event = tracks[event_id] # extract the event from the data
    energies.append(event.energy) # access the neutrino energy of the event, and store it in the energies list
    print(f"Event {idx+1} had energy {event.energy} GeV")

### Now let's look at reconstructing events again, and focus on whether we see any correlations between the energy and how easy it is to reconstruct the event

In [ ]:
reco_game(tracks)

## Did you notice anything ? Does it look like higher energy events would be easier or harder to reconstruct ? Can you think of an intuitive answer for this ?

We can use the `mask_energy` method of the `DataLoader` object to only include energy inbetween `emin` GeV and `emax` GeV. Maybe try to run this analysis again limiting the energy range and note if you see any trends. You can go back to using all the events by using the `unmask` method.

In [ ]:
emin = 5_000
emax = 1000000
# emin = 
# emax = 
tracks.mask_energy(emin, emax)

moon_zeniths = []
moon_azimuths = []

for time in tqdm(tracks["time"]):
    
    zen, az = moon.position(time)
    moon_zeniths.append(zen)
    moon_azimuths.append(az)
    
delta_zeniths = []
for zenith, moon_zenith in zip(tracks["zenith"], moon_zeniths):
    delta_zenith = zenith - moon_zenith
    delta_zeniths.append(delta_zenith)
    
# This will be blank in the final version.
delta_azimuths = []

for azimuth, moon_azimuth in zip(tracks["azimuth"], moon_azimuths):
    
    delta_az = azimuth - moon_azimuth
    if delta_az < -180:
        delta_az += 360
    elif delta_az > 180:
        delta_az -= 360
    delta_azimuths.append(delta_az)

In [ ]:
import numpy as np
minimum_angle = -5 # degrees
maximum_angle = 5 # degrees
n_bins = 20
angle_bins = np.linspace(minimum_angle, maximum_angle, n_bins+1)

h, _, _ = np.histogram2d(
    sine(tracks["zenith"]) * delta_azimuths,
    delta_zeniths,
    bins=[angle_bins, angle_bins]
)

make_plot(h, angle_bins, cmap="cool_r")